# Rapport de TP IA

## Bibiothèques logicielles Python et fichiers intrinsèques
**Préalablement, veillez a installer les fichiers en exécutant le bloc de code suivant.**

In [1]:
!pip install python-sat
!pip install python-constraint
!rm -r Data
!rm -r projet
!rm -r Doc
!unzip Data.zip
!unzip projet.zip
!unzip Doc.zip

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Archive:  Data.zip
   creating: Data/
   creating: Data/pb-etape1/
  inflating: Data/pb-etape1/1-FullIns_3.col  
  inflating: Data/pb-etape1/1-FullIns_4.col  
  inflating: Data/pb-etape1/1-FullIns_5.col  
  inflating: Data/pb-etape1/1-Insertions_4.col  
  inflating: Data/pb-etape1/1-Insertions_5.col  
  inflating: Data/pb-etape1/1-Insertions_6.col  
  inflating: Data/pb-etape1/2-FullIns_3.col  
  inflating: Data/pb-etape1/2-FullIns_4.col  
  inflating: Data/pb-etape1/2-FullIns_5.col  
  inflating: Data/pb-etape1/2-Insertions_3.col  
  inflating: Data/pb-etape1/2-Insertions_4.col  
  inflating: Data/pb-etape1/2-Insertions_5.col  
  inflating: Data/pb-etape1/3-FullIns_3.col  
  inflating: Data/pb-etape1/3-FullIns_4.col  
  inflating: Data/pb-etape1/3-FullIns_5.col  
  inflating: Data/pb-etape1/3-Insertions_3.col  
  inflating: Data/pb

## Contexte du travail à faire

R2D2 est un robot placé dans un monde 2D représenté par un graphe non orienté : les arêtes représentent les routes que R2D2 peut suivre, alors que les sommets représentent les lieux où R2D2 a des choses à faire. Les arêtes seront pondérées pour représenter la longueur du chemin que doit parcourir R2D2 pour aller du sommet origine de l’arête au sommet arrivée de l’arête. Et chaque sommet est pondéré par sa position dans le plan 2D du monde (coordonnées euclidiennes).

On prendra comme hypothèse que R2D2 connaı̂t le monde dans lequel il est placé.
Le travail que doit faire R2D2 : déposer 1 cube de couleur à chaque lieu de manière à ce qu’il y ait dans deux lieux voisins (c-à-d liés par une arête) des cubes de couleur différente. On considérera que R2D2 dispose de suffisamment de cubes.

Le travail de R2D2 va se décomposer en plusieurs tâches :

**Tâche 1**  
Au début, R2D2 décide de n’utiliser que 3 couleurs. Il va donc devoir “raisonner” pour savoir si ces 3 couleurs lui suffisent ou pas pour réaliser son travail.

**Tâche 2**   
Ensuite, R2D2 cherche à savoir comment il va aller déposer les cubes le plus rapidement possible. Pour cela, il cherche des chemins les plus courts en terme de distance parcourue.

**Tâche 3**  
Finalement, R2D2 cherche à savoir combien il lui faut de couleurs a minima pour réaliser son travail.

## Tache 1
Vous devrez identifier le nom et le type du problème, proposer un encodage en logique propositionnelle et le résoudre en utilisant le solver SAT fourni.  
Pour cela vous compléterez la classe *Etape1* du package *etape1* et vous l’exécuterez. Il peut être souhaitable de compléter les tests déjà proposés.

**Note** : Le code ci-dessous a été optimisé et les nom des méthodes, commentaires, ... ont été changés en anglais.

In [2]:
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.solvers.SolverSAT import SolverSAT

class Step1:  # For Task 1
    # ATTRIBUTES
    g: GrapheDeLieux  # The graph representing the world
    base: list  # The clause base representing the problem.
    # It is a list of lists of integers, one integer per variable
    # (positive if positive literal, negative otherwise).
    # Note that 0 is not allowed to represent a variable (updated by updateBase)
    nbVariables: int  # The number of variables used to represent the problem (updated by updateBase)

    # CONSTRUCTOR
    def __init__(self, fn: str, form: bool):
        # :param fn: the filename containing the vertices and edges
        # :param form: used to distinguish between different types of files
        # (for those containing weights and coordinates, form is True; for others, form is False)

        self.g = GrapheDeLieux.loadGraph(fn, form)
        self.base = []  # Base (edge list + node list + color list)
        self.color = []  # Color possibilities list
        self.node = []  # Color possibilities for each node list
        self.edge = []  # Constraints list (graph edges / node links)
        self.nbVariables = 0  # Initial color number

        # Set lists to sorted lists
        self.color.sort()
        self.node.sort()
        self.edge.sort()

    # METHOD
    def updateBase(self, x: int):  # Method to update the clause base and the number of variables based on the problem being solved
        # Initialization
        self.base.clear()
        self.color.clear()
        self.node.clear()
        self.edge.clear()
        self.nbVariables = x * (self.g.getNbSommets())

        # Set up the possible color choices for each state of the graph
        # Example: for 3 colors and a given state, we will have (1001, 1002, 1003)
        # Example: for 4 colors and 2 given states, we will have (1001, 1002, 1003, 1004), (2001, 2002, 2003, 2004)
        for node in range(self.g.getNbSommets()):  # For all graph states
            for color in range(x):  # For all colors
                self.color.extend([(color + 1) + 1000 * (node + 1)])  # Store all color possibilities (up to 999 colors)
                for edge in self.g.getAdjacents(node):  # For all node edges
                    if (1000 + (color + 1) + 1000 * node) != ((edge + 1) * 1000 + (color + 1)):  # If it's not the same node
                        self.edge.append([-(1000 + (color + 1) + 1000 * node),
                                          -((edge + 1) * 1000 + (color + 1))])  # Add the color constraint to the edge list for the node

            self.node.append(self.color)  # Store in a new list for all nodes
            self.color = []  # Empty the color list for the next time

        # Include node, edge, and color constraints in the base list
        self.base.extend(self.node)  # Fill the base with the node list
        self.base.extend(self.edge)  # Fill the base with the edge list

    def runSolver(self):  # Method to call the solver on the clause base representing the problem
        return SolverSAT.solve(self.base)  # :return True if the clause base representing the problem is satisfiable, False otherwise

    def displayBase(self):  # Display the clause base representing the problem
        print('Clause base uses', self.nbVariables, 'variables and contains the following clauses:')
        for clause in self.base:
            print(clause)

class TestStep1:  # Main testing method
    # TESTS
    if __name__ == '__main__':
        step = Step1("Data/town10.txt", True)
        step.updateBase(3)
        print("town10 with 3 colors (expecting True): ", step.runSolver())
        step.updateBase(2)
        print("town10 with 2 colors (expecting False): ", step.runSolver())
        step.updateBase(4)
        print("town10 with 4 colors (expecting True): ", step.runSolver())

        step = Step1("Data/pb-etape1/flat20_3_0.col", False)
        step.updateBase(4)
        print("flat20_3_0.col with 4 colors (expecting True): ", step.runSolver())
        step.updateBase(3)
        print("flat20_3_0.col with 3 colors (expecting True): ", step.runSolver())
        step.updateBase(2)
        print("flat20_3_0.col with 2 colors (expecting False): ", step.runSolver())

        step = Step1("Data/pb-etape1/jean.col", False)
        step.updateBase(10)
        print("jean.col with 10 colors (expecting True): ", step.runSolver())
        step.updateBase(9)
        print("jean.col with 9 colors (expecting False): ", step.runSolver())
        step.updateBase(3)
        print("jean.col with 3 colors (expecting False): ", step.runSolver())

10
town10 with 3 colors (expecting True):  True
town10 with 2 colors (expecting False):  False
town10 with 4 colors (expecting True):  True
20
flat20_3_0.col with 4 colors (expecting True):  True
flat20_3_0.col with 3 colors (expecting True):  True
flat20_3_0.col with 2 colors (expecting False):  False
80
jean.col with 10 colors (expecting True):  True
jean.col with 9 colors (expecting False):  False
jean.col with 3 colors (expecting False):  False


# Etape 2

**Etape 2 = TP 3 à 6** Réalisation de la tâche 2.

Trois cas sont prévus :
* **Cas 1 :** R2D2 commence par calculer le plus court chemin entre deux lieux de son monde. Pour cela, il utilise les distances entre les lieux, ainsi que les coordonnées cartésiennes de chaque lieu.
* **Cas 2 :** Puis il cherche le chemin le plus court permettant de passer dans chaque lieu une seule fois puis de revenir ensuite à son point de départ.
* **Cas 3 :** R2D2 vient d’être “upgradé” : son concepteur l’a équipé de la capacité à voler. Il peut désormais relier en ligne droite chacun des lieux de son monde sans être obligé de suivre les routes. Il peut donc trouver un autre chemin plus court permettant de passer par chaque lieu et de revenir ensuite à son point de départ.

Dans les trois cas, vous devrez identifier le nom et le type du problème, proposer un mode de représentation et le résoudre en utilisant un des algorithmes fournis. Pour cela vous compléterez les classes *EtatCas1*, *EtatCas2* et *EtatCas3* du package *etape2* et vous les utiliserez pour compléter et exécuter la classe *Etape2* du package *etape2*. Il peut être souhaitable de compléter les tests déjà proposés.

### Cas 1
R2D2 calcule le plus court chemin entre deux lieux de son monde en utilisant les distances entre les lieux, ainsi que les coordonnées cartésiennes de chaque lieu.

In [3]:
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.etape2.Etat import Etat
from projet.solvers.SolverAStar import SolverAStar

class EtatCas1(Etat) : # Classe pour definir un etat pour le cas 1 de la tache 2 (hérite de Etat)
    # ATTRIBUTS
    tg : GrapheDeLieux # le graphe representant le monde

    # CONSTRUCTEUR
    def __init__(self, tg : GrapheDeLieux, etat_debut=None, etat_final=None) :
        self.tg            = tg
        self.etat_courant  = etat_debut
        self.etat_final    = etat_final

    # METHODES issues de Etat
    def estSolution(self) : # Methode detectant si l'etat est une solution :return true si l'etat courant est une solution, false sinon
        return self.etat_courant == self.etat_final

    def successeurs(self) : # Methode permettant de recuperer la liste des etats successeurs de l'etat courant :return liste des etats successeurs de l'etat courant
        tab = []
        for adjacent in self.tg.getAdjacents(self.etat_courant):
          tab.append(EtatCas1(self.tg,adjacent, self.etat_final))
        return tab

    def h(self) :   # Methode permettant de recuperer l'heuristique de l'etat courant :return heuristique de l'etat courant
        return GrapheDeLieux.dist(self.etat_courant, self.etat_final , self.tg)

    def k(self, e) : # Methode permettant de recuperer le cout du passage de l'etat courant à l'etat e
        return self.tg.getCoutArete(self.etat_courant, e)

    def displayPath(self, pere) :
        chemin = [self.etat_final]
        courant = self.etat_final
        while courant != None:
            courant = pere[courant]
            chemin.append(courant)
        print("Sur",self.tg.getNbSommets(),"villes de",chemin[-2]," à ",chemin[0]," : [",", ".join(str(etat) for etat in chemin[:-1]),"]")

    # METHODES pour pouvoir utiliser cet objet dans des listes et des map
    def __hash__(self) : # Methode permettant de recuperer le code de hachage de l'etat courant pour une utilisation dans des tables de hachage -> return code de hachage
        return hash(self.etat_courant)

    def __eq__(self, o) :# Methode de comparaison de l'etat courant avec l'objet o -> return true si l'etat courant et o sont egaux, false sinon
        return self.etat_courant == o

    # METHODES pour affichage futur (heritee d'Object)
    def __str__(self) : # Methode mettant l'etat courant sous la forme d'une chaine de caracteres en prevision d'un futur affichage
        return str(self.etat_courant)
    
tg = GrapheDeLieux.loadGraph("Data/town10.txt",True)
cas1 = EtatCas1(tg,0,9)
SolverAStar.aStar(cas1)

cas1 = EtatCas1(tg,5,9)
SolverAStar.aStar(cas1)

cas1 = EtatCas1(tg,2,9)
SolverAStar.aStar(cas1)

cas1 = EtatCas1(tg,1,7)
SolverAStar.aStar(cas1)

tg = GrapheDeLieux.loadGraph("Data/town30.txt",True)
cas1 = EtatCas1(tg,0,25)
SolverAStar.aStar(cas1)

tg = GrapheDeLieux.loadGraph("Data/town150.txt",True)
cas1 = EtatCas1(tg,0,145)
SolverAStar.aStar(cas1)

tg = GrapheDeLieux.loadGraph("Data/town1000.txt",True)
cas1 = EtatCas1(tg,0,997)
SolverAStar.aStar(cas1)

10
Sur 10 villes de 0  à  9  : [ 9, 8, 4, 1, 0 ]
la lg du plus court chemin est  1190.971503412202
9
=======(nb d'etats explores = 7)========
=======(nb d'etats generes = 21)========

Sur 10 villes de 5  à  9  : [ 9, 8, 6, 5 ]
la lg du plus court chemin est  858.6177055898913
9
=======(nb d'etats explores = 5)========
=======(nb d'etats generes = 16)========

Sur 10 villes de 2  à  9  : [ 9, 8, 6, 3, 2 ]
la lg du plus court chemin est  1090.639192762493
9
=======(nb d'etats explores = 10)========
=======(nb d'etats generes = 31)========

Sur 10 villes de 1  à  7  : [ 7, 6, 3, 1 ]
la lg du plus court chemin est  889.1949043390727
7
=======(nb d'etats explores = 5)========
=======(nb d'etats generes = 14)========

26
Sur 26 villes de 0  à  25  : [ 25, 24, 19, 16, 12, 5, 3, 0 ]
la lg du plus court chemin est  1856.5
25
=======(nb d'etats explores = 20)========
=======(nb d'etats generes = 76)========

146
Sur 146 villes de 0  à  145  : [ 145, 133, 111, 89, 69, 45, 5, 0 ]
la lg du plus cou

### Cas 2
R2D2 cherche le chemin le plus court permettant de passer dans chaque lieu une seule fois puis de revenir ensuite à son point de départ.

In [4]:
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.etape2.Etat import Etat
from projet.solvers.SolverAStar import SolverAStar

class EtatCas2(Etat):
    tg: GrapheDeLieux  # Le graphe représentant le monde

    # CONSTRUCTEUR
    def __init__(self, tg: GrapheDeLieux, etat_visit=None, etat_courant=0, etat_debut=0):
        self.tg = tg
        self.etat_courant = etat_courant
        self.etat_debut = etat_debut

        # Ajout de l'état courant dans l'ensemble des états qui ont déjà été visités
        if etat_visit is None:
            self.etat_visit = []
        else:
            self.etat_visit = etat_visit
        self.etat_visit.append(etat_courant)

    # METHODES issues de Etat
    def estSolution(self):
        # Vérifie si tous les états ont été visités et si le point de départ est également le point d'arrivée
        return len(self.etat_visit) == self.tg.getNbSommets() + 1 and self.etat_debut == self.etat_courant

    def successeurs(self):
        tableau = []

        # Si tous les états ont été visités et que le point de départ est adjacent au dernier état visité,
        # alors on crée un état supplémentaire pour revenir au point de départ
        if len(self.etat_visit) == self.tg.getNbSommets() and (self.etat_debut in self.tg.getAdjacents(self.etat_courant)):
            return [EtatCas2(self.tg, self.etat_visit.copy(), self.etat_debut, self.etat_debut)]

        # Ajoute tous les états adjacents non visités dans le tableau
        for adjacent in self.tg.getAdjacents(self.etat_courant):
            if adjacent not in self.etat_visit:
                tableau.append(EtatCas2(self.tg, self.etat_visit.copy(), adjacent, self.etat_debut))

        return tableau

    def h(self):
        # Heuristique basée sur le nombre d'états restants à visiter et le poids minimum entre les états sur terre
        return (self.tg.getNbSommets() - len(self.etat_visit)) * self.tg.getPoidsMinTerre()

    def k(self, e):
        # Coût entre l'état courant et l'état e
        return self.tg.getCoutArete(self.etat_courant, e.etat_courant)

    def displayPath(self, _):
        # Affiche les états visités dans l'ordre
        print("sur", self.tg.getNbSommets(), "villes :", self.etat_visit)

    # METHODES pour pouvoir utiliser cet objet dans des listes et des map
    def __hash__(self):
        # Utilise un tuple pour générer un hash basé sur les états visités
        # Dans le cas du EtatCas2, les états visités sont stockés dans une liste, et les listes en Python ne sont pas hashables 
        # parce qu'elles sont mutables (leur contenu peut être modifié après la création). 
        # Cependant, les tuples sont immuables, ce qui signifie que leur contenu ne peut pas être modifié après leur création. 
        # La conversion en tuple dans la fonction __hash__ est une manière de contourner cette limitation. 
        # En convertissant la liste d'états visités en un tuple (qui est hashable), on peut utiliser cette 
        # valeur comme clé dans un dictionnaire ou comme élément dans un ensemble.
        return hash(tuple(self.etat_visit))

    def __eq__(self, o):
        # Compare les états visités pour vérifier l'égalité
        return self.etat_courant == o

    # METHODES pour affichage futur (héritée d'Object)
    def __str__(self):
        # Affiche les états visités sous forme de chaîne de caractères
        return " ".join(str(etat) for etat in self.etat_visit)

# CAS 2 : tour complet par voie de terre
tg = GrapheDeLieux.loadGraph("Data/town10.txt", True)
cas2 = EtatCas2(tg)
SolverAStar.aStar(cas2)

10
sur 10 villes : [0, 1, 3, 4, 8, 9, 7, 6, 5, 2, 0]
la lg du plus court chemin est  3792.190362007193
0 1 3 4 8 9 7 6 5 2 0
=======(nb d'etats explores = 330)========
=======(nb d'etats generes = 341)========



### Cas 3 
R2D2 vient d’être “upgradé” : son concepteur l’a équipé de la capacité à voler. Il peut désormais relier en ligne droite chacun des lieux de son monde sans être obligé de suivre les routes. Il peut donc trouver un autre chemin plus court permettant de passer par chaque lieu et de revenir ensuite à son point de départ.

In [5]:
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.etape2.Etat import Etat

class EtatCas3(Etat):
    tg: GrapheDeLieux  # Le graphe représentant le monde

    # CONSTRUCTEUR
    def __init__(self, tg: GrapheDeLieux, etat_visit=None, etat_courant=0, etat_debut=0):
        self.tg = tg
        self.etat_courant = etat_courant
        self.etat_debut = etat_debut

        # Ajout de l'état courant dans l'ensemble des états qui ont déjà été visités
        if etat_visit is None:
            self.etat_visit = []
        else:
            self.etat_visit = etat_visit
        self.etat_visit.append(etat_courant)

    # METHODES issues de Etat
    def estSolution(self):
        # Vérifie si tous les états ont été visités et si le point de départ est également le point d'arrivée
        return len(self.etat_visit) == self.tg.getNbSommets() + 1 and self.etat_debut == self.etat_courant

    def successeurs(self):
        tableau = []

        # Si tous les états ont été visités, crée un état supplémentaire pour revenir au point de départ
        if len(self.etat_visit) == self.tg.getNbSommets():
            return [EtatCas3(self.tg, self.etat_visit.copy(), self.etat_debut, self.etat_debut)]

        # Ajoute tous les sommets non visités dans le tableau (pas de condition supplémentaire ici par rapport au cas 2)
        for sommet in self.tg.getSommets():
            if sommet not in self.etat_visit:
                tableau.append(EtatCas3(self.tg, self.etat_visit.copy(), sommet, self.etat_debut))

        return tableau

    def h(self):
        # Heuristique basée sur le nombre d'états restants à visiter et le poids minimum dans les airs
        return (self.tg.getNbSommets() - len(self.etat_visit)) * self.tg.getPoidsMinAir()

    def k(self, e):
        # Coût entre l'état courant et l'état e, calculé en utilisant la distance entre les états
        return GrapheDeLieux.dist(self.etat_courant, e.etat_courant, self.tg)

    def displayPath(self):
        # Affiche le chemin trouvé en utilisant une flèche pour représenter les déplacements entre les états
        print("Le chemin trouvé est : ")
        print(" >>>>> ".join(str(etat) for etat in self.etat_visit))

    # METHODES pour pouvoir utiliser cet objet dans des listes et des map
    def __hash__(self):
        # Utilise un tuple pour générer un hash basé sur le graphe, l'état courant et les états visités
        return hash((self.tg,self.etat_courant, tuple(self.etat_visit)))

    def __eq__(self, o):
        # Compare les états visités pour vérifier l'égalité
        return self.etat_visit == o

    # METHODES pour affichage futur (héritée d'Object)
    def __str__(self):
        # Affiche les états visités sous forme de chaîne de caractères
        return " ".join(str(etat) for etat in self.etat_visit)
    
# CAS 3 : tour complet par voie des airs
# Utilisation de la fonction aStarOpti car chargement très gourmant en ressources
tg = GrapheDeLieux.loadGraph("Data/town6.txt",True)
cas3 = EtatCas3(tg)
SolverAStar.aStarOpti(cas3)

tg = GrapheDeLieux.loadGraph("Data/town7.txt",True)
cas3 = EtatCas3(tg)
SolverAStar.aStarOpti(cas3)

tg = GrapheDeLieux.loadGraph("Data/town8.txt",True)
cas3 = EtatCas3(tg)
SolverAStar.aStarOpti(cas3)

tg = GrapheDeLieux.loadGraph("Data/town9.txt",True)
cas3 = EtatCas3(tg)
SolverAStar.aStarOpti(cas3)

tg = GrapheDeLieux.loadGraph("Data/town10.txt",True)
cas3 = EtatCas3(tg)
SolverAStar.aStarOpti(cas3)

tg = GrapheDeLieux.loadGraph("Data/town11.txt",True)
cas3 = EtatCas3(tg)
SolverAStar.aStarOpti(cas3)


6
Le chemin trouvé est : 
0 >>>>> 1 >>>>> 2 >>>>> 3 >>>>> 5 >>>>> 4 >>>>> 0
la lg du plus court chemin est  1360.6495955560758
0 1 2 3 5 4 0
=======(nb d'etats explores = 131)========
=======(nb d'etats generes = 238)========

7
Le chemin trouvé est : 
0 >>>>> 1 >>>>> 2 >>>>> 3 >>>>> 6 >>>>> 5 >>>>> 4 >>>>> 0
la lg du plus court chemin est  1638.459980067224
0 1 2 3 6 5 4 0
=======(nb d'etats explores = 552)========
=======(nb d'etats generes = 1110)========

8
Le chemin trouvé est : 
0 >>>>> 4 >>>>> 5 >>>>> 7 >>>>> 6 >>>>> 3 >>>>> 2 >>>>> 1 >>>>> 0
la lg du plus court chemin est  1729.6228205017967
0 4 5 7 6 3 2 1 0
=======(nb d'etats explores = 1430)========
=======(nb d'etats generes = 3514)========

9
Le chemin trouvé est : 
0 >>>>> 1 >>>>> 2 >>>>> 3 >>>>> 6 >>>>> 7 >>>>> 5 >>>>> 8 >>>>> 4 >>>>> 0
la lg du plus court chemin est  1855.2162397331167
0 1 2 3 6 7 5 8 4 0
=======(nb d'etats explores = 4796)========
=======(nb d'etats generes = 13342)========

10
Le chemin trouvé est : 


KeyboardInterrupt: 

In [60]:
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.solvers.SolverAStar import SolverAStar
# rajouter ensuite le import permettant d'utiliser le solver (l'algo) choisi

class Etape2 : # Classe pour realiser les tests de l'etape 2 du projet (execution des cas 1, 2 et 3 : tache 2)
    if __name__ == '__main__':
        

SyntaxError: incomplete input (2634818480.py, line 7)

# Etape 3

**Etape 3 = TP 7 et 8** Réalisation de la tâche 2 (suite).

R2D2 se rend compte que sa méthode précédente met trop de temps à s’exécuter ! Du coup, il renonce à trouver le chemin le plus court et est prêt à tenter des chemins un peu moins bons pourvu qu’il arrive à les calculer plus vite. Et comme il est curieux, il va essayer deux méthodes différentes pour voir celle qui est la plus efficace. Vous devrez proposer un mode de représentation et résoudre le problème en utilisant au moins deux des algorithmes fournis.
Pour cela vous compléterez la classe *UneSolution* du package *etape3* et vous l’utiliserez pour compléter et exécuter la classe *Etape3* du package *etape3*. Il peut être souhaitable de compléter les tests déjà proposés.

In [36]:
from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.etape3.Solution import Solution
import random

class UneSolution(Solution): # Classe pour définir une solution pour le cas 3 de la tâche 2 (hérite de Solution)
    # attributs
    tg: GrapheDeLieux
    """ le graphe représentant le monde """

    # constructeurs
    def __init__(self, tg: GrapheDeLieux, cycle=None):
        """ constructeur d'une solution à partir du graphe représentant le monde
        :param tg: graphe représentant le monde """
        self.tg = tg

        if cycle==None:
            self.cycle=[i for i in range(self.tg.getNbSommets())]
            random.shuffle(self.cycle)
            self.cycle.append(self.cycle[0])
        else:
            self.cycle=cycle

    # méthodes de la classe abstraite Solution
    def lesVoisins(self):
        """ méthode récupérant la liste des voisins de la solution courante
        :return liste des voisins de la solution courant"""
        voisin=[]
        list_i=[]
        for i in range(self.tg.getNbSommets()-1):
            index=random.randint(1,self.tg.getNbSommets()-1)
            index2=random.randint(index+1,self.tg.getNbSommets())
            while index in list_i:
                index=random.randint(1,self.tg.getNbSommets()-1)
                index2=random.randint(index+1,self.tg.getNbSommets())
            list_i.append(index)
            nouveau_cycle=self.cycle[0:index] + self.cycle[index2-1:index-1:-1] + self.cycle[index2:]
            voisin.append(UneSolution(self.tg,nouveau_cycle))
        return voisin

    def unVoisin(self):
        """methode recuperant un voisin de la solution courante
        :return voisin de la solution courante"""
        return [self.lesVoisins()[0]]

    def eval(self):
        """ méthode récupérant la valeur de la solution courante
        :return valeur de la solution courante """
        val=0
        for i in range(len(self.cycle)-1):
            etat1=self.cycle[i]
            etat2=self.cycle[i+1]
            val+=GrapheDeLieux.dist(etat1,etat2,self.tg)
        return val

    def nelleSolution(self):
        """ méthode générant aléatoirement une nouvelle solution à partir de la solution courante
        :return nouvelle solution générée aléatoirement à partir de la solution courante"""
        cycle=[i for i in range(self.tg.getNbSommets())]
        random.shuffle(cycle)
        new_cycle=cycle+[cycle[0]]
        return UneSolution(self.tg,new_cycle)

    def displayPath(self):
        """ méthode affichant la solution courante comme un chemin dans le graphe """
        print("la solution courante : ",self.eval(),"\n",self.cycle,"\n")

    # méthodes pour pouvoir utiliser cet objet dans des listes et des map
    def __hash__(self):
        """ méthode permettant de récupérer le code de hachage de la solution courante
        pour une utilisation dans des tables de hachage
        :return code de hachage"""
        return hash(self.cycle)

    def __eq__(self, o):
        """ méthode de comparaison de la solution courante avec l'objet o
        :param o: l'objet avec lequel on compare
        :return True si la solution courante et o sont égaux, False sinon"""
        return self.eval == o

    # méthode pour affichage futur (héritée d'Object)
    def __str__(self):
        """ méthode mettant la solution courante sous la forme d'une
        chaîne de caractères en prévision d'un futur affichage
        :return représentation de la solution courante sous la forme d'une chaîne de caractère"""
        return " ".join(str(sommet) for sommet in self.cycle)


In [37]:
"""
module principal pour l'etape 3
"""

from projet.outils.GrapheDeLieux import GrapheDeLieux
from projet.solvers.SolverHC import SolverHC
from projet.solvers.SolverTabou import SolverTabou

class Etape3:
    """  classe pour realiser les tests de l'etape 3 du projet (suite tache 2) """
    """  methode de TESTS pour Etape3
    """
    if __name__ == '__main__':

        #    cas 1 : 10 villes de 0 à 9
        tg: GrapheDeLieux = GrapheDeLieux.loadGraph("Data/town10.txt", True)
        tsp: UneSolution = UneSolution(tg)
        print("======== Solver 1 pour 10 villes de 0 a 9 : \n")
        SolverHC.hilClimbing2(tsp, 100)  # Changer le nombre d'essais si nécessaire
        print("\n======== Solver 2 pour 10 villes de 0 a 9 : \n")
        SolverTabou.tabou(tsp, 100)  # Changer le nombre d'essais si nécessaire

        #    cas 2 : 26 villes de 0 à 25
        tg = GrapheDeLieux.loadGraph("Data/town30.txt", True)
        tsp = UneSolution(tg)
        print("\n======== Solver 1 pour 26 villes de 0 a 25 : \n")
        SolverHC.hilClimbing2(tsp, 100)  # Changer le nombre d'essais si nécessaire
        print("\n======== Solver 2 pour 26 villes de 0 a 25 : \n")
        SolverTabou.tabou(tsp, 100)  # Changer le nombre d'essais si nécessaire

        #    cas 3 : 150 villes
        tg = GrapheDeLieux.loadGraph("Data/town150.txt", True)
        tsp = UneSolution(tg)
        print("\n======== Solver 1 pour 150 villes : \n")
        SolverHC.hilClimbing2(tsp, 100)  # Changer le nombre d'essais si nécessaire
        print("\n======== Solver 2 pour 150 villes : \n")
        SolverTabou.tabou(tsp, 100)  # Changer le nombre d'essais si nécessaire)

        #    cas 4 : 1000 villes
        tg = GrapheDeLieux.loadGraph("Data/town1000.txt", True)
        tsp = UneSolution(tg)
        print("\n======== Solver 1 pour 1000 villes : \n")
        SolverHC.hilClimbing2(tsp, 100)  # Changer le nombre d'essais si nécessaire
        print("\n======== Solver 2 pour 1000 villes : \n")
        SolverTabou.tabou(tsp, 100)  # Changer le nombre d'essais si nécessaire


10
======== Solver 1 pour 10 villes de 0 a 9 : 

HC avec plusieurs essais et tirage aleatoire d'une nelle solution pour chaque essai

La meilleure solution obtenue : 3 5 9 7 6 2 0 1 4 8 3=======(arret apres 100 essais : nb d'etats explores = 136)========


======== Solver 2 pour 10 villes de 0 a 9 : 

	 Il y a 92 tabou(s) :

La meilleure solution obtenue : 5 8 9 4 0 1 2 3 6 7 5=======(arret apres 100 essais : nb d'etats explores = 100)========

26

======== Solver 1 pour 26 villes de 0 a 25 : 

HC avec plusieurs essais et tirage aleatoire d'une nelle solution pour chaque essai

La meilleure solution obtenue : 8 12 11 19 17 3 2 0 4 13 20 18 5 15 1 14 16 6 9 7 22 10 23 21 24 25 8=======(arret apres 100 essais : nb d'etats explores = 181)========


======== Solver 2 pour 26 villes de 0 a 25 : 

	 Il y a 59 tabou(s) :

La meilleure solution obtenue : 2 3 1 4 8 6 9 12 16 13 17 20 19 25 24 21 23 22 18 14 15 11 10 7 5 0 2=======(arret apres 100 essais : nb d'etats explores = 200)========

146

# Etape 4

**Etape 4 = TP 9 et 10** Réalisation de la tâche 3.

Vous devrez identifier le nom et le type du problème, proposer un encodage sous la forme d’un graphe de contraintes et le résoudre en utilisant un des algorithmes fournis. Pour cela vous compléterez et exécuterez la classe *Etape4* du package *etape4*. Il peut être souhaitable de compléter les tests déjà proposés.

In [ ]:
"""
module principal pour l'etape 4
"""

from projet.outils.GrapheDeLieux import GrapheDeLieux

# rajouter ensuite le import permettant d'utiliser le solver (l'algo) choisi
# from projet.solvers.... import ...

class Etape4 :
    """
    classe de test pour l'etape 4
    """

    if __name__ == '__main__':

        print("\n========== TEST  ==========")
        print("=============================")

        #    TEST 1 : town10.txt avec 3 couleurs
        tg : GrapheDeLieux = GrapheDeLieux.loadGraph("Data/town10.txt",True)
        print("\nTest sur town10 avec 3 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer

        #    TEST 2 : town10.txt avec 2 couleurs
        print("\nTest sur town10 avec 2 couleurs (on attend NOK) :")
        #    choisir ici un algo et l'executer

        #    TEST 3 : town10.txt avec 4 couleurs
        print("\nTest sur town10 avec 4 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer



        #    TEST 4 : flat20_3_0.col avec 4 couleurs
        tg = GrapheDeLieux.loadGraph("Data/pb-etape1/flat20_3_0.col",False)
        print("Test sur flat20_3_0.col avec 4 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer

        #    TEST 5 : flat20_3_0.col avec 3 couleurs
        print("Test sur flat20_3_0.col avec 3 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer

        #    TEST 6 : flat20_3_0.col avec 2 couleurs
        print("Test sur flat20_3_0.col avec 2 couleurs (on attend NOK) :")
        #    choisir ici un algo et l'executer



        #    TEST 7 : jean.col avec 10 couleurs
        tg = GrapheDeLieux.loadGraph("Data/pb-etape1/jean.col",False)
        print("Test sur jean.col avec 10 couleurs (on attend OK) :")
        #    choisir ici un algo et l'executer

        #    TEST 9 : jean.col avec 3 couleurs
        print("Test sur jean.col avec 3 couleurs (on attend NOK) :")
        #    choisir ici un algo et l'executer

        #    TEST 8 : jean.col avec 9 couleurs
        print("Test sur jean.col avec 9 couleurs (on attend NOK) :")
        #    choisir ici un algo et l'executer






# Etape 5

**Etape 5 = TP 11 et 12** Réalisation de la tâche 2 (suite et fin).

Comme R2D2 aime aussi beaucoup les maths et qu’il veut épater ses concepteurs, il reprend le cas 3 de la tâche 2, et cherche à le résoudre en l’exprimant à l’aide de formules mathématiques. Vous devrez proposer un encodage approprié utilisant le langage ZIMPL et résoudre le problème en utilisant le solver SCIP.

**Installation de l'environnement de travail**


*   récupération de l'outil SCIP sur votre machine (voir le README dans la documentation sur ZIMPL et sur SCIP qui se trouve dans le fichier *Doc.zip* que vous pouvez télécharger depuis la liste des fichiers et installer en le décompressant sur votre machine.)
*   compression du répertoire SCIP sur votre machine (celui qui contient les 3 sous-répertoire "bin", "lib" et "include") pour créer le fichier *SCIP.zip*
*   puis avec le bloc de code suivant : upload du fichier *SCIP.zip* sur GoogleCoLab, décompression et test avec le code ZIMPL donné





In [1]:
!unzip SCIP.zip
!SCIP/bin/scip -f projet/etape5/test.zpl

Archive:  SCIP.zip
   creating: SCIP/
   creating: SCIP/lib/
    linking: SCIP/lib/libscip.so     -> libscip.so.8.0 
  inflating: SCIP/lib/libsoplex.a    
  inflating: SCIP/lib/libclusol.a    
  inflating: SCIP/lib/libzimpl-pic.a  
  inflating: SCIP/lib/libsoplexshared.so.6.0.3.0  
  inflating: SCIP/lib/libzimpl.a     
    linking: SCIP/lib/libgcg.so      -> libgcg.so.3.5 
  inflating: SCIP/lib/libpapilo-core.a  
  inflating: SCIP/lib/libbliss.a     
    linking: SCIP/lib/libsoplexshared.so.6.0  -> libsoplexshared.so.6.0.3.0 
   creating: SCIP/lib/cmake/
   creating: SCIP/lib/cmake/papilo/
  inflating: SCIP/lib/cmake/papilo/papilo-targets-release.cmake  
  inflating: SCIP/lib/cmake/papilo/FindTBB.cmake  
  inflating: SCIP/lib/cmake/papilo/papilo-config-version.cmake  
  inflating: SCIP/lib/cmake/papilo/FindQuadmath.cmake  
  inflating: SCIP/lib/cmake/papilo/papilo-targets.cmake  
  inflating: SCIP/lib/cmake/papilo/papilo-config.cmake  
   creating: SCIP/lib/cmake/zimpl/
  inflating: SC

**Exemple de code ZIMPL utilisé pour le test précédent :** il correspond à l'exemple vu en cours (résultat attendu : x1 = 4, x2 = 0 pour une valeur de 68)


> var x1 integer >= 0 ;

> var x2 integer >= 0 ;

> maximize res : (17 * x1) + (12 * x2) ;

> subto c1 : (10 * x1) + (7 * x2) <= 40 ;

> subto c2 : x1 + x2 <= 5 ;




**Le travail à faire**

Ecrire un programme ZIMPL répondant à la question posée. Ce programme sera à placer et à tester sur le modèle du bloc de code suivant.

Si vous définissez plusieurs programmes ZIMPL, vous devrez faire un bloc de code par programme. Attention à bien choisir le nom des fichiers de sauvegarde pour ne pas en écraser.


In [ ]:
from google.colab import files

contenuFichierZIMPL = """
    var x1 integer >= 0 ;
    var x2 integer >= 0 ;
    maximize res : (17 * x1) + (12 * x2) ;
    subto c1 : (10 * x1) + (7 * x2) <= 40 ;
    subto c2 : x1 + x2 <= 5 ;
    """

with open('projet/etape5/fichier1.zpl', 'w') as f:
  f.write(contenuFichierZIMPL)

!SCIP/bin/scip -f projet/etape5/fichier1.zpl

